In [1]:
!pip install japanize_matplotlib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pathlib import Path
from glob import glob

from sklearn.preprocessing import LabelEncoder, StandardScaler

import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor as mor
from sklearn.metrics import mean_squared_error as MAE
from sklearn.model_selection import KFold
import japanize_matplotlib

path_start = Path("/content/drive/MyDrive/Nishika/Mansion_2023_spring")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv(path_start/"input/df_train.csv", index_col=0)
test = pd.read_csv(path_start/"input/test.csv", index_col=0)
sample = pd.read_csv(path_start/"input/sample_submission.csv")

df_ConCat = pd.concat([train, test])
print(df_ConCat.shape)

(784713, 28)


## 特徴量の削除

In [4]:
Fvalue = lambda x: df_ConCat.drop(x, axis=1)
List_del = ["取引時点", "市区町村名", "間取り", "種類",
            "前面道路：方位", "前面道路：種類",	"前面道路：幅員（ｍ）", "地域", "土地の形状",
            "間口", "建物の構造", "延床面積（㎡）", "用途", "取引の事情等",
            "最寄駅：名称", "建築年"]

for c in List_del:
  df_ConCat = df_ConCat.drop(c, axis=1)
display(df_ConCat.shape)
df_ConCat.head()

(784713, 12)

,ID,市区町村コード,都道府県名,地区名,最寄駅：距離（分）,面積（㎡）,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,改装,取引価格（総額）_log
0,8067540,8217,茨城県,野々井,13,70,NaN,第１種住居地域,60.0,200.0,未改装,6.977724
1,8027265,8219,茨城県,ひたち野東,8,70,住宅,第２種中高層住居専用地域,60.0,200.0,未改装,7.255273
2,8061526,8220,茨城県,苅間,4,105,NaN,商業地域,80.0,400.0,未改装,7.556303
3,8086147,8220,茨城県,並木,75,100,NaN,第１種中高層住居専用地域,60.0,200.0,未改装,7.380211
4,8049498,8224,茨城県,ひがし野,8,95,住宅,近隣商業地域,80.0,200.0,未改装,7.518514


## 標準化

In [5]:
List_std = ["最寄駅：距離（分）", "面積（㎡）", "建ぺい率（％）", "容積率（％）"]
scaler = StandardScaler()
scaler.fit(df_ConCat[List_std])
df_ConCat[List_std] = scaler.transform(df_ConCat[List_std])
df_ConCat.head()

,ID,市区町村コード,都道府県名,地区名,最寄駅：距離（分）,面積（㎡）,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,改装,取引価格（総額）_log
0,8067540,8217,茨城県,野々井,0.144049,0.431679,NaN,第１種住居地域,-0.741730,-0.695601,未改装,6.977724
1,8027265,8219,茨城県,ひたち野東,-0.266625,0.431679,住宅,第２種中高層住居専用地域,-0.741730,-0.695601,未改装,7.255273
2,8061526,8220,茨城県,苅間,-0.595164,1.726269,NaN,商業地域,1.177623,0.650409,未改装,7.556303
3,8086147,8220,茨城県,並木,5.236408,1.541328,NaN,第１種中高層住居専用地域,-0.741730,-0.695601,未改装,7.380211
4,8049498,8224,茨城県,ひがし野,-0.266625,1.356387,住宅,近隣商業地域,1.177623,-0.695601,未改装,7.518514


## 特徴量追加

In [6]:
for i in glob(str(path_start/"LoE/*.csv")):
  df_ConCat = pd.merge(df_ConCat, pd.read_csv(i), on="ID", how="left")

for i in glob(str(path_start/"TargetEncoding/*.csv")):
  df_ConCat = pd.merge(df_ConCat, pd.read_csv(i), on="ID", how="left")

for i in glob(str(path_start/"feature_value/*.csv")):
  df_ConCat = pd.merge(df_ConCat, pd.read_csv(i), on="ID", how="left")
df_ConCat.shape

(784713, 374)

## 特徴量を50個に絞る

In [7]:
list_handred = pd.read_csv(path_start/"my_list_50.csv", header=None).values.tolist()
flattened = [x for row in list_handred for x in row]
display(len(flattened))
display(flattened)

50

['地区名',
 '建築年',
 '面積（㎡）',
 '都道府県名',
 '市区町村名_TE面積（㎡）',
 '最寄駅：距離（分）',
 '市区町村コード',
 '改装',
 '取引時点_TE取引時期_年四半期',
 '市区町村名_TE最寄駅：距離（分）',
 '間取り_TE取引時期_四半期',
 '市区町村名_TE取引時期_四半期',
 '取引時点_TE取引時期_年',
 '市区町村名_TE容積率（％）',
 'ID',
 '取引時点_TE建築年',
 '最寄駅：名称_TE容積率（％）',
 '地区名_TE最寄駅：距離（分）',
 '市区町村名_TE取引時期_年',
 '市区町村名_TE取引時期_年四半期',
 '市区町村名_TE取引-建築',
 '今後の利用目的',
 '取引の事情等_TE取引時期_四半期',
 '都市計画',
 '取引時点_TE面積（㎡）',
 '都道府県名_TE面積（㎡）',
 '市区町村名_TE建築年',
 '容積率（％）',
 '取引時点_TE取引-建築',
 '最寄駅：名称_TE建ぺい率（％）',
 '取引時点_TE最寄駅：距離（分）',
 '取引の事情等_TE建築年',
 '最寄駅：名称_TE取引時期_四半期',
 '市区町村名_TE建ぺい率（％）',
 '取引時点_TE建ぺい率（％）',
 '地区名_TE建築年',
 '取引の事情等_TE容積率（％）',
 '最寄駅：名称_TE取引-建築',
 '建物の構造_TE最寄駅：距離（分）',
 '都市計画_TE最寄駅：距離（分）',
 '間取り_TE取引時期_年',
 '最寄駅：名称_TE最寄駅：距離（分）',
 '取引の事情等_TE取引時期_年',
 '地区名_TE容積率（％）',
 '最寄駅：名称_TE取引時期_年四半期',
 '取引時点_TE容積率（％）',
 '地区名_TE取引-建築',
 '最寄駅：名称_TE面積（㎡）',
 '取引の事情等_TE面積（㎡）',
 '今後の利用目的_TE面積（㎡）']

In [8]:
flattened.remove("ID")

In [9]:
df_ConCat2 = pd.concat([df_ConCat[["ID", "取引価格（総額）_log"]], df_ConCat[flattened]], axis=1)
df_ConCat2.head()

,ID,取引価格（総額）_log,地区名,建築年,面積（㎡）,都道府県名,市区町村名_TE面積（㎡）,最寄駅：距離（分）,市区町村コード,改装,...,間取り_TE取引時期_年,最寄駅：名称_TE最寄駅：距離（分）,取引の事情等_TE取引時期_年,地区名_TE容積率（％）,最寄駅：名称_TE取引時期_年四半期,取引時点_TE容積率（％）,地区名_TE取引-建築,最寄駅：名称_TE面積（㎡）,取引の事情等_TE面積（㎡）,今後の利用目的_TE面積（㎡）
0,8067540,6.977724,野々井,1996.0,0.431679,茨城県,0.201441,0.144049,8217,未改装,...,-0.042201,-0.051169,0.01425,-0.433236,-0.096591,-0.102286,0.026846,0.105338,-0.002202,0.025406
1,8027265,7.255273,ひたち野東,1998.0,0.431679,茨城県,0.506946,-0.266625,8219,未改装,...,-0.042201,-0.519268,0.01425,0.203528,0.046625,0.028981,-0.704945,0.641592,-0.002202,0.008770
2,8061526,7.556303,苅間,2009.0,1.726269,茨城県,1.006775,-0.595164,8220,未改装,...,-0.042201,-0.587841,0.01425,0.306295,0.463295,-0.036798,-0.888053,1.280655,-0.002202,0.025406
3,8086147,7.380211,並木,2007.0,1.541328,茨城県,1.006775,5.236408,8220,未改装,...,-0.103538,1.064501,0.01425,-0.416577,-0.048201,-0.043977,0.196723,0.967881,-0.002202,0.025406
4,8049498,7.518514,ひがし野,2009.0,1.356387,茨城県,0.690122,-0.266625,8224,未改装,...,-0.103538,-0.262707,0.01425,-0.690946,0.420120,0.019880,-0.926995,0.736961,-0.002202,0.008770


## ラベルエンコーディング

In [10]:
List_Encode = []
for i, t in enumerate(df_ConCat2.dtypes):
  if t!="float64" and t!="int64":
    List_Encode.append(df_ConCat2.columns[i])

List_Encode

['地区名', '都道府県名', '改装', '今後の利用目的', '都市計画']

In [11]:
for c in List_Encode:
  le = LabelEncoder()
  le.fit(df_ConCat2[c])
  df_ConCat2[c] = le.transform(df_ConCat2[c])

In [12]:
List_plus = ["用途_住宅", "用途_不明", "用途_店舗",	"用途_その他", "用途_事務所",
             "用途_駐車場", "用途_倉庫", "用途_作業場", "用途_工場", "建物の構造_ＲＣ",
             "建物の構造_鉄骨造", "建物の構造_ＳＲＣ", "建物の構造_不明", "建物の構造_木造", "建物の構造_軽量鉄骨造",
             "建物の構造_ブロック造", "間取り_数字", "利用変異", "取引の事情等_不明",	"取引の事情等_調停・競売等",
             "取引の事情等_瑕疵有りの可能性",	"取引の事情等_関係者間取引", "取引の事情等_その他事情有り", "取引の事情等_他の権利・負担付き",
             "利用一致_不明", "利用一致_住宅", "利用一致_その他", "利用一致_事務所", "利用一致_店舗",
             "市区町村コード", "ID"]
list_and = list(set(list(df_ConCat2.columns)).intersection(set(List_plus)))
display(type(list_and))
display(list_and)

list

['市区町村コード', 'ID']

## 分割

In [13]:
if df_ConCat2["取引価格（総額）_log"].isna().sum()!=0:
  test = df_ConCat2[df_ConCat2["取引価格（総額）_log"].isna()==True]
  train = df_ConCat2[df_ConCat2["取引価格（総額）_log"].isna()!=True]
else:
  test = df_ConCat2[df_ConCat2["取引価格（総額）_log"]==0]
  train = df_ConCat2[df_ConCat2["取引価格（総額）_log"]!=0]


In [14]:
aim = "取引価格（総額）_log"
df_y = train[aim]
df_x = train.drop([aim], axis=1)
test = test.drop([aim], axis=1)
df_x.head()

,ID,地区名,建築年,面積（㎡）,都道府県名,市区町村名_TE面積（㎡）,最寄駅：距離（分）,市区町村コード,改装,取引時点_TE取引時期_年四半期,...,間取り_TE取引時期_年,最寄駅：名称_TE最寄駅：距離（分）,取引の事情等_TE取引時期_年,地区名_TE容積率（％）,最寄駅：名称_TE取引時期_年四半期,取引時点_TE容積率（％）,地区名_TE取引-建築,最寄駅：名称_TE面積（㎡）,取引の事情等_TE面積（㎡）,今後の利用目的_TE面積（㎡）
0,8067540,11184,1996.0,0.431679,38,0.201441,0.144049,8217,1,-1.580274,...,-0.042201,-0.051169,0.01425,-0.433236,-0.096591,-0.102286,0.026846,0.105338,-0.002202,0.025406
1,8027265,80,1998.0,0.431679,38,0.506946,-0.266625,8219,1,0.784888,...,-0.042201,-0.519268,0.01425,0.203528,0.046625,0.028981,-0.704945,0.641592,-0.002202,0.008770
2,8061526,9896,2009.0,1.726269,38,1.006775,-0.595164,8220,1,-0.666462,...,-0.042201,-0.587841,0.01425,0.306295,0.463295,-0.036798,-0.888053,1.280655,-0.002202,0.025406
3,8086147,822,2007.0,1.541328,38,1.006775,5.236408,8220,1,-0.397693,...,-0.103538,1.064501,0.01425,-0.416577,-0.048201,-0.043977,0.196723,0.967881,-0.002202,0.025406
4,8049498,79,2009.0,1.356387,38,0.690122,-0.266625,8224,1,0.731134,...,-0.103538,-0.262707,0.01425,-0.690946,0.420120,0.019880,-0.926995,0.736961,-0.002202,0.008770


In [15]:
test.head()

,ID,地区名,建築年,面積（㎡）,都道府県名,市区町村名_TE面積（㎡）,最寄駅：距離（分）,市区町村コード,改装,取引時点_TE取引時期_年四半期,...,間取り_TE取引時期_年,最寄駅：名称_TE最寄駅：距離（分）,取引の事情等_TE取引時期_年,地区名_TE容積率（％）,最寄駅：名称_TE取引時期_年四半期,取引時点_TE容積率（％）,地区名_TE取引-建築,最寄駅：名称_TE面積（㎡）,取引の事情等_TE面積（㎡）,今後の利用目的_TE面積（㎡）
765412,1000077,4280,2016.0,0.616621,4,0.365548,-0.841569,1101,1,1.591193,...,-0.042201,0.286794,0.01425,0.59565,-0.110204,0.089514,0.474869,0.236322,-0.002202,-0.565428
765413,1000081,4280,2012.0,0.616621,4,0.365548,-0.677299,1101,1,1.591193,...,0.055225,0.286794,0.01425,0.59565,-0.110204,0.089514,0.474869,0.236322,-0.002202,0.008770
765414,1000128,4280,1984.0,0.061796,4,0.365548,-0.759434,1101,1,1.644947,...,0.055225,0.017191,0.01425,0.59565,-0.063107,0.114934,0.474869,0.325373,-0.002202,0.008770
765415,1000129,4280,1986.0,-1.232794,4,0.365548,-0.759434,1101,1,1.644947,...,-0.054852,0.017191,0.01425,0.59565,-0.063107,0.114934,0.474869,0.325373,-0.002202,0.008770
765416,1000130,4280,2016.0,0.616621,4,0.365548,-0.841569,1101,1,1.644947,...,-0.042201,0.017191,0.01425,0.59565,-0.063107,0.114934,0.474869,0.325373,-0.002202,0.008770


## 学習

In [16]:
kf = KFold(n_splits=5, shuffle=True, random_state=71)

# ハイパーパラメータの調整
params = {'boosting_type': 'gbdt',
          "objective": "regression_l1",
          "reg_alpha": 0.1,
          "reg_lambda": 0.1,
          "random_state": 71,
          "max_depth": 10,
          "n_estimators": 10000,
          "colsample_bytree": 1.0,
          "colsample_bylevel": 0.3384286948321845,
          "lambda_l1": 0.0001,
          'metrics': 'mae',
          'learning_rate': 0.1,
          'min_child_weight': 6.8344456043713215}
num_round = 500

In [17]:
scores = []
Preds = []
models = []
i = 0
for tr_idx, va_idx in kf.split(df_x):
  i += 1
  print("{}回目".format(i))
  x_df_train, x_df_val = df_x.iloc[tr_idx], df_x.iloc[va_idx]
  y_df_train, y_df_val = df_y.iloc[tr_idx], df_y.iloc[va_idx]
  lgb_train = lgb.Dataset(x_df_train, y_df_train)
  lgb_val = lgb.Dataset(x_df_val, y_df_val)

  evaluation_results = {}
  model = lgb.train(params, lgb_train,
                    num_boost_round=num_round,
                    valid_sets=[lgb_train, lgb_val],
                    categorical_feature=List_Encode+list_and,
                    evals_result=evaluation_results,   # 学習の経過を保存
                    early_stopping_rounds=100,          # アーリーストッピング
                    verbose_eval=100)
  models.append(model)

  va_pred = model.predict(x_df_val)
  score = MAE(np.array(y_df_val), va_pred, squared=False)
  scores.append(score)

print(f'MAE: {np.mean(scores):.4f}')
print(scores)

1回目


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['ID', '今後の利用目的', '地区名', '市区町村コード', '改装', '都市計画', '都道府県名']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a futur

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] lambda_l1 is set=0.0001, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.0001
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] lambda_l1 is set=0.0001, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.0001
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.464541 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14767
[LightGBM] [Info] Number of data points in the train set: 612329, nu

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 7.278754
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.0805946	valid_1's l1: 0.0832697
[200]	training's l1: 0.0745534	valid_1's l1: 0.0787237
[300]	training's l1: 0.0720696	valid_1's l1: 0.0771148
[400]	training's l1: 0.0706885	valid_1's l1: 0.0762811
[500]	training's l1: 0.0698128	valid_1's l1: 0.0758084
[600]	training's l1: 0.0692699	valid_1's l1: 0.0755429
[700]	training's l1: 0.0688307	valid_1's l1: 0.0753267
[800]	training's l1: 0.0684499	valid_1's l1: 0.0751458
[900]	training's l1: 0.0681217	valid_1's l1: 0.0749844
[1000]	training's l1: 0.0678289	valid_1's l1: 0.0748623
[1100]	training's l1: 0.0676025	valid_1's l1: 0.0747963
[1200]	training's l1: 0.0673991	valid_1's l1: 0.0747193
[1300]	training's l1: 0.0671779	valid_1's l1: 0.0746522
[1400]	training's l1: 0.0669934	valid_1's l1: 0.0745974
[1500]	training's l1: 0.066813	valid_1's l1: 0.0745366
[1600]	training's l1: 0.0666232	valid_1's l1: 0.0744567

...

[LightGBM] [Info] Start training from score 7.278754
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.080762	valid_1's l1: 0.0828369
[200]	training's l1: 0.0748605	valid_1's l1: 0.0782122
[300]	training's l1: 0.0724572	valid_1's l1: 0.0765818
[400]	training's l1: 0.0710164	valid_1's l1: 0.0757137
[500]	training's l1: 0.0701299	valid_1's l1: 0.0752108
[600]	training's l1: 0.0695439	valid_1's l1: 0.0749289
[700]	training's l1: 0.0690733	valid_1's l1: 0.0746875
[800]	training's l1: 0.0687396	valid_1's l1: 0.0745437
[900]	training's l1: 0.0684117	valid_1's l1: 0.0744027
[1000]	training's l1: 0.0681064	valid_1's l1: 0.0742621
[1100]	training's l1: 0.0678285	valid_1's l1: 0.0741441
[1200]	training's l1: 0.06757	valid_1's l1: 0.0740342
[1300]	training's l1: 0.0673411	valid_1's l1: 0.0739544
[1400]	training's l1: 0.067123	valid_1's l1: 0.0738615
[1500]	training's l1: 0.0669192	valid_1's l1: 0.0737775
[1600]	training's l1: 0.0667591	valid_1's l1: 0.0737145
[1

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['ID', '今後の利用目的', '地区名', '市区町村コード', '改装', '都市計画', '都道府県名']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a futur

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] lambda_l1 is set=0.0001, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.0001
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] lambda_l1 is set=0.0001, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.0001
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.242094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14797
[LightGBM] [Info] Number of data points in the train set: 612330, nu

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 7.278754
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.0806895	valid_1's l1: 0.0830185
[200]	training's l1: 0.074786	valid_1's l1: 0.0784321
[300]	training's l1: 0.0723432	valid_1's l1: 0.0767761
[400]	training's l1: 0.0709334	valid_1's l1: 0.0759399
[500]	training's l1: 0.0700256	valid_1's l1: 0.0754544
[600]	training's l1: 0.0693948	valid_1's l1: 0.0751334
[700]	training's l1: 0.0689378	valid_1's l1: 0.0749176
[800]	training's l1: 0.0685811	valid_1's l1: 0.0747512
[900]	training's l1: 0.0682585	valid_1's l1: 0.0746053
[1000]	training's l1: 0.0679938	valid_1's l1: 0.0744995
[1100]	training's l1: 0.0677614	valid_1's l1: 0.0744163
[1200]	training's l1: 0.0675123	valid_1's l1: 0.074316
[1300]	training's l1: 0.0673055	valid_1's l1: 0.0742422
[1400]	training's l1: 0.0671061	valid_1's l1: 0.0741778
[1500]	training's l1: 0.0669633	valid_1's l1: 0.0741279
[1600]	training's l1: 0.0668131	valid_1's l1: 0.0740831


In [18]:
# 予測値を保存するデータフレームの初期化
df_preds = pd.DataFrame()

# 保存してあるモデルで予測
for i, model in enumerate(models):
    y_pred = model.predict(test)
    df_preds[i] = y_pred

 # 保存した予測値を表示
df_preds.head()

,0,1,2,3,4
0,7.687252,7.682464,7.669007,7.674060,7.713570
1,7.694255,7.696383,7.708959,7.678437,7.721641
2,7.137371,7.121927,7.115191,7.105992,7.120146
3,6.605401,6.616861,6.624890,6.580849,6.582642
4,7.654864,7.662820,7.676149,7.637772,7.669898


In [19]:
FileName = "sub_lightGBM_10_mean"

sample[aim] = df_preds.mean(axis=1)
sample.to_csv(path_start/"output/{}.csv".format(FileName), index = False)
pd.read_csv(path_start/"output/{}.csv".format(FileName))

,ID,取引価格（総額）_log
0,1000077,7.685271
1,1000081,7.699935
2,1000128,7.120126
3,1000129,6.602129
4,1000130,7.660301
...,...,...
19296,47006020,7.559805
19297,47006229,7.555365
19298,47006331,7.589384
19299,47006332,7.252445
